In [ ]:
!pip install oauthlib
!pip install requests_oauthlib

In [ ]:
import os
import json
import shutil

import requests
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

In [ ]:
# Get your client credentials from sentinel hub
client_id = os.environ['SH_ID']
client_secret = os.environ['SH_SECRET']

client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)
token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_id=client_id, client_secret=client_secret)
resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo")

In [ ]:
template_json = {
    "input": {
        "bounds": {
            "bbox": [
                13.822174072265625,
                45.85080395917834,
                14.55963134765625,
                46.29191774991382
            ]
        },
        "data": [{
            "type": "S2L2A",
            'dataFilter': {
                'mosaickingOrder': 'leastCC',
                'maxCloudCoverage': 3,
                "timeRange": {
                    "from": "2018-10-01T00:00:00Z",
                    "to": "2019-12-31T00:00:00Z"
                }
            }
        }],
        'bands': ['B02', 'B03', 'B04'], # RGB
        "output": {
            "width": 256,
            "height": 256
        }
    },
    "evalscript": """
    //VERSION=3

    function setup() {
      return {
        input: ["B02", "B03", "B04"],
        output: {
          bands: 3
        }
      };
    }

    function evaluatePixel(
      sample,
      scenes,
      inputMetadata,
      customData,
      outputMetadata
    ) {
      return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02];
    }
    """
}

In [ ]:
def download_image(lat, long, path):
    pad = 0.05
    bbox = [long-pad, lat-pad, long+pad, lat+pad]
    data = template_json
    data['input']['bounds']['bbox'] = bbox
    response = oauth.post('https://services.sentinel-hub.com/api/v1/process', json=data)
    print(response)
    with open(path, 'wb') as f:
        f.write(response.content)        

In [ ]:
# Test it out by downloading Venice
download_image(lat=45.44, long=12.33, path='test_image_venice.png')